In [1]:
import numpy as np
import pandas as pd

from src.M4_data import prepare_M4_data
from src.M4_experiment import plot_model_prediction, evaluate_model_prediction
from src.ESRNN import ESRNN

In [2]:
X_train_df, y_train_df, X_test_df, y_test_df = prepare_M4_data('Quarterly', num_obs=3) #, num_obs=5
y_train_df

,unique_id,ds,y
0,Q1,1970-01-02,7407.412314
1,Q1,1970-01-03,7528.566074
2,Q1,1970-01-04,7374.709225
3,Q1,1970-01-05,7395.514848
4,Q1,1970-01-06,7654.007989
...,...,...,...
70,Q3,1970-01-22,7212.253565
71,Q3,1970-01-23,7246.218675
72,Q3,1970-01-24,7178.380550
73,Q3,1970-01-25,7182.867128


In [3]:
import torch

## Weekly
# esrnn = ESRNN(max_epochs=4, batch_size=4, dilations=[[1, 4], [52]], rnn_weight_decay=0.5, freq_of_test=1)

## Quarterly
# esrnn = ESRNN(max_epochs=1, batch_size=3, rnn_weight_decay=0.5, freq_of_test=1)

## Daily
# esrnn = ESRNN(max_epochs=2, batch_size=16, learning_rate=3e-4, per_series_lr_multip=1.5,
#               gradient_eps=1e-6, level_variability_penalty=100,
#               dilations=[[1,7],[28]], add_nl_layer=True,
#               seasonality=7, input_size=7, output_size=14)

## Debugging
with torch.autograd.detect_anomaly():
    esrnn = ESRNN(max_epochs=4, max_periods=1, batch_size=3, state_hsize=4,
                  seasonality=1, input_size=3, output_size=8, dilations=[[1, 2]],
                  rnn_weight_decay=1.0, freq_of_test=1, random_seed=1, cell_type='ResLSTM')
    esrnn.fit(X_train_df, y_train_df, X_test_df, y_test_df)

=============== Training ESRNN  ===============

previous dilated_inputs torch.Size([2, 3, 4]) 
tensor([[[ 0.0769,  0.0439,  0.0568,  1.0000],
         [ 0.0765,  0.0520,  0.0725,  1.0000],
         [ 0.0777,  0.0523,  0.0651,  1.0000]],

        [[ 0.0371,  0.0533, -0.0436,  1.0000],
         [ 0.0406,  0.0613, -0.0565,  1.0000],
         [ 0.0249,  0.0375, -0.0680,  1.0000]]], grad_fn=<CatBackward>)
hidden torch.Size([1, 3, 4]) torch.Size([1, 3, 4])
tensor([[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]])
tensor([[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]])
outputs.size() [2, 3, 4]
new dilated_inputs torch.Size([2, 3, 4]) 
tensor([[[ 0.0769,  0.0439,  0.0568,  1.0000],
         [ 0.0765,  0.0520,  0.0725,  1.0000],
         [ 0.0777,  0.0523,  0.0651,  1.0000]],

        [[ 0.0371,  0.0533, -0.0436,  1.0000],
         [ 0.0406,  0.0613, -0.0565,  1.0000],
         [ 0.0249,  0.0375, -0.0680,  1.0000]]], grad_fn=<CatBackward>)
lo

In [ ]:
plot_model_prediction(y_train_df, X_test_df, y_test_df, model=esrnn, u_id='Q1')

In [ ]:
esrnn.evaluate_model_prediction(y_train_df, X_test_df, y_test_df)

In [ ]:
(0.778*(23.0/100) + 0.847*(24.0/100) + 0.836*(48.0/100) + 0.920*(5.0/100))

In [ ]:
#(13.176*() + 9.679*() + 12.126*())
(14.42*(23.0/(23+24+48)) + 10.09*(24.0/(23+24+48)) + 10.81*(48.0/(23+24+48)))

In [ ]:
y_test_df